# Capstone Project - The Battle of the Neighborhoods (Week 4)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction / Business Problem <a name="introduction"></a>

In this project we will try to find possible locations for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Japanese restaurant** in **Madrid**, Spain.

Since there are lots of restaurants in Madrid we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Japanese restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met. 

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decision are:
* number of existing restaurants in the neighboorhood (any type of restaurant)
* number of and distance to Japanese restaurants in the neighboorhood, if any
* distance of neighboorhood to city center


The following data sources will be needed to extract/generate the required information:
* location and limits of each district will be obtained using geopy and an open-source json file
* number of restaurants and their type and location in every neighboorhood will be obtained using **Foursquare API**
* coordinate of Madrid center will be obtained using **Geopy** of well known Madrid location (Kilómetro Cero - Puerta del Sol, 7, 28013 Madrid, Spain).


### Import and Access


Given that we will be using several libraries and API's, the following cells are related with their import and access.

In [101]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import math # library for random number generation
import pickle

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium --yes
import folium # plotting library
from folium.plugins import HeatMap


# libraries for converting to a cartesian 2D coordinates system
#!conda install -c conda-forge shapely --yes 
import shapely.geometry
#!conda install -c conda-forge pyproj --yes 
import pyproj


print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [102]:
client_id= 'JDCEXFAV4G2QWJWEG4ACBPFJEYFM214UNZARDLVPSCEAHH0U' # your Foursquare ID
client_secret = 'L0NBAAC3QSFV4DVAMXKEAHEDDTLTF0YOMUU3A0ZJ2HUWGK5U' # your Foursquare Secret
version = '20180604'
limit = 30
print('Your credentails:')
print('CLIENT_ID: ' + client_id)
print('CLIENT_SECRET:' + client_secret)

Your credentails:
CLIENT_ID: JDCEXFAV4G2QWJWEG4ACBPFJEYFM214UNZARDLVPSCEAHH0U
CLIENT_SECRET:L0NBAAC3QSFV4DVAMXKEAHEDDTLTF0YOMUU3A0ZJ2HUWGK5U


### Creation of the map

Using Geopy, let's find the latitude and longitude of the central point of Madrid, Kilómetro Cero. For this, we can use its address, Puerta del Sol, 7, 28013 Madrid, Spain 

In [120]:
def get_coordinates(user_agent, address):
    try:
        geolocator = Nominatim(user_agent=user_agent)
        location = geolocator.geocode(address)
        lat = location.latitude
        lon = location.longitude
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Puerta del Sol, 7, 28013 Madrid, Spain' # address of Kilómetro Cero
madrid_center = get_coordinates("foursquare_agent", address)
print('Coordinates of {}: {}'.format(address, madrid_center))

Coordinates of Puerta del Sol, 7, 28013 Madrid, Spain: [40.4163403, -3.7034386198258615]


Now, we can use a json file and geopy to find the limits of each district.

In [121]:
madrid_districts_url = 'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/madrid-districts.geojson'
madrid_districts = requests.get(madrid_districts_url).json()

districts_data = madrid_districts['features']
names = []
lats = []
lons = []


for data in districts_data:
    name = data['properties']['name']
    district_latlon = data['geometry']['coordinates']
    district_latlon = district_latlon[0][0]
    
    tmp_lats = []
    tmp_lons = []
    for i in range (len(district_latlon)):
        tmp_lats.append(district_latlon[i][1])
        tmp_lons.append(district_latlon[i][0])
    
    names.append(name)
    lats.append(np.mean(tmp_lats))
    lons.append(np.mean(tmp_lons))


def districts_style(feature):
    return { 'color': 'blue', 'fill': False}

In order to place a marker on each district, let's put the name, the average latitude and the average longitude of each district in a dataframe.

In [122]:
# define the dataframe columns
column_names = ['District', "Latitude", "Longitude"] 

# instantiate the dataframe
districts = pd.DataFrame()

districts['District'] = names
districts['Latitude'] = lats
districts['Longitude'] = lons

districts

,District,Latitude,Longitude
0,Centro,40.418309,-3.704902
1,Arganzuela,40.401842,-3.698664
2,Retiro,40.410488,-3.677238
3,Salamanca,40.433693,-3.671852
4,Chamartin,40.467394,-3.675737
5,Tetuan,40.468377,-3.701191
6,Chamberi,40.439211,-3.706746
7,Fuencarral-El Pardo,40.557248,-3.745399
8,Moncloa-Aravaca,40.460135,-3.766995
9,Latina,40.392334,-3.777401


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~3km from Kilometro Cero. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [123]:
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Madrid center longitude={}, latitude={}'.format(madrid_center[1], madrid_center[0]))
x, y = lonlat_to_xy(madrid_center[1], madrid_center[0])
print('Madrid center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Madrid center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Madrid center longitude=-3.7034386198258615, latitude=40.4163403
Madrid center UTM X=-1090934.9917725557, Y=4645641.410147375
Madrid center longitude=-3.7034386198258593, latitude=40.4163403


Let's create a hexagonal grid of cells: we offset every other row, and adjust vertical row spacing so that every cell center is equally distant from all it's neighbors.

In [124]:
madrid_center_x, madrid_center_y = lonlat_to_xy(madrid_center[1], madrid_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = madrid_center_x - 3000
x_step = 600
y_min = madrid_center_y - 3000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(madrid_center_x, madrid_center_y, x, y)
        if (distance_from_center <= 3001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

90 candidate neighborhood centers generated.


Let's put the information found in a dataframe.

In [125]:
df_locations = pd.DataFrame({'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})


df_locations.head(5)

,Latitude,Longitude,X,Y,Distance from center
0,40.390921,-3.710178,-1.092135e+06,4.642926e+06,2969.072921
1,40.392045,-3.703482,-1.091535e+06,4.642926e+06,2781.257631
2,40.393168,-3.696787,-1.090935e+06,4.642926e+06,2715.767665
3,40.394291,-3.690090,-1.090335e+06,4.642926e+06,2781.257631
4,40.395413,-3.683394,-1.089735e+06,4.642926e+06,2969.072921


Finally, we let's make a map. Also, place radius of 1km, 2km, 3km around Kilómetro Cero.

In [148]:
map_madrid = folium.Map(location=madrid_center, zoom_start=12.4)
folium.TileLayer('cartodbpositron').add_to(map_madrid) #cartodbpositron cartodbdark_matter
folium.Marker(madrid_center).add_to(map_madrid)
folium.Circle(madrid_center, radius=1000, fill=False, color='red').add_to(map_madrid)
folium.Circle(madrid_center, radius=2000, fill=False, color='red').add_to(map_madrid)
folium.Circle(madrid_center, radius=3000, fill=False, color='red').add_to(map_madrid)

for i in range(1,len(districts.index)):
    folium.Marker([districts.Latitude[i], districts.Longitude[i]], popup=districts.District[i]).add_to(map_madrid)
    
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=300, color='yellow', fill=False).add_to(map_madrid)
        
folium.GeoJson(madrid_districts, style_function=districts_style, name='geojson').add_to(map_madrid)
map_madrid

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~3km from Kilometro Cero. 

### Foursquare API
Now let's use Foursquare API to get info on restaurants in each district.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Japanese restaurant' category, as we need info on Japanese restaurants in the district.

In [127]:
# Category IDs corresponding to japanese restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

japanese_restaurant_categories = ['4bf58dd8d48988d111941735','55a59bace4b013909087cb0c',
                                  '55a59bace4b013909087cb30', '55a59bace4b013909087cb21', 
                                  '55a59bace4b013909087cb06','55a59bace4b013909087cb1b',
                                  '55a59bace4b013909087cb1e','55a59bace4b013909087cb18',
                                  '55a59bace4b013909087cb24','55a59bace4b013909087cb15',
                                  '55a59bace4b013909087cb27','55a59bace4b013909087cb12',
                                  '4bf58dd8d48988d1d2941735','55a59bace4b013909087cb2d',
                                  '55a59a31e4b013909087cb00','55a59af1e4b013909087cb03',
                                  '55a59bace4b013909087cb2a','55a59bace4b013909087cb0f',
                                  '55a59bace4b013909087cb33','55a59bace4b013909087cb09',
                                  '55a59bace4b013909087cb36']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, version, radius=500, limit=100):
    version = version
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [128]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found japanese restaurants

def get_restaurants(lats, lons):
    restaurants = {}
    japanese_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, client_id, client_secret, version, radius=500, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_japanese = is_restaurant(venue_categories, specific_filter=japanese_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_japanese, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_japanese:
                    japanese_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, japanese_restaurants, location_restaurants

restaurants, japanese_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)


Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


Let's organise the data in dataframes easily manipulated.

In [129]:
tmp_names = []
tmp_lats = []
tmp_lons = []
tmp_adresses = []

for key in restaurants.keys():
    tmp_names.append(restaurants[key][1])
    tmp_lats.append(restaurants[key][2])
    tmp_lons.append(restaurants[key][3])
    tmp_adresses.append(restaurants[key][4])
    
df_restaurants = pd.DataFrame()

df_restaurants["Name"] = tmp_names
df_restaurants["Latitude"] = tmp_lats
df_restaurants["Longitude"] = tmp_lons
df_restaurants["Adresses"] = tmp_adresses

df_restaurants.head(5)

,Name,Latitude,Longitude,Adresses
0,Bar El muro,40.391612,-3.710201,"Madrid Madrid, España"
1,Restaurante Oviedo,40.394803,-3.709216,"Antonio López, 55, Madrid Madrid, España"
2,Open Market Cafe 25,40.393703,-3.707733,"28019 Madrid Madrid, España"
3,Los Cigarrales,40.395286,-3.710397,"C/ Antonio López 52, 28019 Madrid Madrid, España"
4,Villa Rosa,40.394494,-3.707858,"C/ Antonio López 63, 28019 Madrid Madrid, España"


In [130]:
tmp_names = []
tmp_lats = []
tmp_lons = []
tmp_adresses = []

for key in japanese_restaurants.keys():
    tmp_names.append(japanese_restaurants[key][1])
    tmp_lats.append(japanese_restaurants[key][2])
    tmp_lons.append(japanese_restaurants[key][3])
    tmp_adresses.append(japanese_restaurants[key][4])
    
df_restaurants_jap = pd.DataFrame()

df_restaurants_jap["Name"] = tmp_names
df_restaurants_jap["Latitude"] = tmp_lats
df_restaurants_jap["Longitude"] = tmp_lons
df_restaurants_jap["Adresses"] = tmp_adresses

df_restaurants_jap.head(5)

,Name,Latitude,Longitude,Adresses
0,Japan Pearl,40.392100,-3.692415,"C. Zinc (C. Bolívar, 8), 28045 Madrid Madrid, ..."
1,Maki Mendez Alvaro,40.396320,-3.684876,"C. Mezquite, 2 (C. Méndez Alvaro), 28045 Madri..."
2,Sakura,40.396072,-3.712003,"Calle Antonio López, 35. (Pasaje Montserrat), ..."
3,sushi raku,40.404623,-3.708216,"Gasometro 11, Madrid Madrid, España"
4,Zen Central,40.405812,-3.670820,España


Let's now see all the collected restaurants (green) in our area of interest on map, and let's also show Japanese restaurants in a different color (red).

In [131]:
for i in range(0,len(df_restaurants.index)):
    folium.CircleMarker([df_restaurants.Latitude[i], df_restaurants.Longitude[i]], radius=3,
                        color="green", fill=True, fill_color="green", fill_opacity=1).add_to(map_madrid)
    
for i in range(0,len(df_restaurants_jap.index)):
    folium.CircleMarker([df_restaurants_jap.Latitude[i], df_restaurants_jap.Longitude[i]], radius=3,
                        color="red", fill=True, fill_color="red", fill_opacity=1).add_to(map_madrid)
    
    
map_madrid

The data phase, which includes its gathering and treatment, is now completed. We can move to the methodology. 

## Methodology <a name="methodology"></a>

We intend to select the best district or districts for a Japanese restaurant in Madrid.

For this, we want locations which fulfill as best as possible the following conditions:
* have few restaurants around the candidate location;
* have few japanese restaurants around the candidate location;
* is close to the center of the city;

In this project we will direct our efforts on detecting areas of Madrid that have low restaurant density, particularly those with low number of Japanese restaurants. We will limit our analysis to area ~5km around city center.

In first step we have collected the required **data: location and type (category) of every restaurant within 5km from Madrid center** (Kilometro Cero). We have also **identified Japanese restaurants** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**restaurant density**' across different areas of Madrid - we will use **heatmaps** to identify a few promising areas close to center with low number of restaurants in general (*and* no Japanese restaurants in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than two restaurants in radius of 250 meters**, and we want locations **without Japanese restaurants in radius of 400 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's make an overview of the data.

In [132]:
print('Total number of restaurants:', len(df_restaurants.index))
print('Total number of Japanese restaurants:', len(df_restaurants_jap.index))
print('Percentage of Japanese restaurants: {:.2f}%'.format(len(df_restaurants_jap) / len(df_restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 1469
Total number of Japanese restaurants: 78
Percentage of Japanese restaurants: 5.31%
Average number of restaurants in neighborhood: 11.133333333333333


In [133]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(5)

Average number of restaurants in every area with radius=300m: 11.133333333333333


,Latitude,Longitude,X,Y,Distance from center,Restaurants in area
0,40.390921,-3.710178,-1.092135e+06,4.642926e+06,2969.072921,1
1,40.392045,-3.703482,-1.091535e+06,4.642926e+06,2781.257631,5
2,40.393168,-3.696787,-1.090935e+06,4.642926e+06,2715.767665,13
3,40.394291,-3.690090,-1.090335e+06,4.642926e+06,2781.257631,6
4,40.395413,-3.683394,-1.089735e+06,4.642926e+06,2969.072921,9


OK, now let's calculate the distance to nearest Japanese restaurant from every area candidate center (not only those within 300m - we want distance to closest one, regardless of how distant it is).

In [134]:
distances_to_jap_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in japanese_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_jap_restaurant.append(min_distance)

df_locations['Distance to Japanese restaurant'] = distances_to_jap_restaurant

df_locations.head(5)

,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to Japanese restaurant
0,40.390921,-3.710178,-1.092135e+06,4.642926e+06,2969.072921,1,610.931038
1,40.392045,-3.703482,-1.091535e+06,4.642926e+06,2781.257631,5,876.802163
2,40.393168,-3.696787,-1.090935e+06,4.642926e+06,2715.767665,13,401.673517
3,40.394291,-3.690090,-1.090335e+06,4.642926e+06,2781.257631,6,322.933883
4,40.395413,-3.683394,-1.089735e+06,4.642926e+06,2969.072921,9,166.129920


In [135]:
print('Average distance to closest Japanese restaurant from each area center:', df_locations['Distance to Japanese restaurant'].mean())

Average distance to closest Japanese restaurant from each area center: 481.18127036803


OK, so on average a Japanese restaurant can be found within ~500m from every area center candidate. That's fairly close, so we need to filter our areas carefully!

Let's crete a map showing the density of restaurants and try to extract some meaningful info from that. Also, a few circles indicating distance of 1km, 2km and 3km from Kilometro Cero.

In [136]:
map_madrid = folium.Map(location=madrid_center, zoom_start=13)

for i in range(1,len(districts.index)):
    folium.Marker([districts.Latitude[i], districts.Longitude[i]], popup=districts.District[i]).add_to(map_madrid)
    
folium.Circle(madrid_center, radius=1000, fill=False, color='white').add_to(map_madrid)
folium.Circle(madrid_center, radius=2000, fill=False, color='white').add_to(map_madrid)
folium.Circle(madrid_center, radius=3000, fill=False, color='white').add_to(map_madrid)

for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=300, color='yellow', fill=False).add_to(map_madrid)
        
folium.GeoJson(madrid_districts, style_function=districts_style, name='geojson').add_to(map_madrid)

HeatMap(df_restaurants.drop(["Name", "Adresses"], axis=1).values.tolist()).add_to(map_madrid)

map_madrid

Using the heatmap, we can see that within 2km to the center of the city, that there are some pockets in Centro, Arganzuela, Retiro, Latina and Moncloa-Aravaca.  

Now, let's analyse the density of Japanese restaurants in Madrid.

In [137]:
map_madrid = folium.Map(location=madrid_center, zoom_start=13)
folium.Marker(madrid_center, popup='Centro (Kilometro Cero)').add_to(map_madrid)

for i in range(1,len(districts.index)):
    folium.Marker([districts.Latitude[i], districts.Longitude[i]], popup=districts.District[i]).add_to(map_madrid)
    
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=300, color='yellow', fill=False).add_to(map_madrid)
        
folium.GeoJson(madrid_districts, style_function=districts_style, name='geojson').add_to(map_madrid)

HeatMap(df_restaurants_jap.drop(["Name", "Adresses"], axis=1).values.tolist()).add_to(map_madrid)

folium.Circle(madrid_center, radius=1000, fill=False, color='white').add_to(map_madrid)
folium.Circle(madrid_center, radius=2000, fill=False, color='white').add_to(map_madrid)
folium.Circle(madrid_center, radius=3000, fill=False, color='white').add_to(map_madrid)

map_madrid

There are not many Japanese restaurants in Madrid, which is great news for our stakeholders. The districts observed in the last heatmap are still good options. 

As such, we have four districts which can be eligible candidates, Centro, Arganzuela, Retiro, Latina and Moncloa-Aravaca. Given that the Centro, Arganzuela and Retiro have closest pockets to the center, let's consider only these ones.

### Centro, Arganzuela and Retiro

Let's define new, more narrow region of interest, which will include low-restaurant-count parts within 2km.

In [138]:
roi_x_min = madrid_center_x - 2000
roi_y_max = madrid_center_y + 1000
roi_width = 5000
roi_height = 5000
roi_center_x = roi_x_min + 2500
roi_center_y = roi_y_max - 2500
roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]

Let's also create new, more dense grid of location candidates restricted to our new region of interest (let's make our location candidates 100m appart).

In [139]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 100
y_step = 100 * k 
roi_y_min = roi_center_y - 2000

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 2001):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

1453 candidate neighborhood centers generated.


OK. Now let's calculate two most important things for each location candidate: number of restaurants in vicinity (we'll use radius of 250 meters) and distance to closest Japanese restaurant.

In [140]:
def count_restaurants_nearby(x, y, restaurants, radius=250):    
    count = 0
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_restaurant(x, y, restaurants):
    d_min = 100000
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_restaurant_counts = []
roi_jap_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_restaurants_nearby(x, y, restaurants, radius=250)
    roi_restaurant_counts.append(count)
    distance = find_nearest_restaurant(x, y, japanese_restaurants)
    roi_jap_distances.append(distance)
print('done.')

Generating data on location candidates... done.


Let's put this into a dataframe.

In [141]:
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Restaurants nearby':roi_restaurant_counts,
                                 'Distance to Japanese restaurant':roi_jap_distances})
ds = []
for i in range (len(df_roi_locations.index)):
    x, y = lonlat_to_xy(df_roi_locations.Longitude[i], df_roi_locations.Latitude[i])
    tmp = calc_xy_distance(madrid_center_x, madrid_center_y, x, y)
    ds.append(tmp)
    
df_roi_locations["Distance form center"] = ds   

df_roi_locations = df_roi_locations[df_roi_locations["Distance form center"]<=2000]


df_roi_locations.head(10)

,Latitude,Longitude,X,Y,Restaurants nearby,Distance to Japanese restaurant,Distance form center
509,40.398935,-3.703705,-1.091385e+06,4.643700e+06,2,761.472919,1992.631404
510,40.399122,-3.702589,-1.091285e+06,4.643700e+06,3,799.395960,1972.455300
511,40.399309,-3.701473,-1.091185e+06,4.643700e+06,5,847.482623,1957.186734
512,40.399497,-3.700357,-1.091085e+06,4.643700e+06,5,904.112654,1946.941168
513,40.399684,-3.699241,-1.090985e+06,4.643700e+06,7,913.687099,1941.798113
514,40.399871,-3.698125,-1.090885e+06,4.643700e+06,10,866.506814,1941.798113
515,40.400058,-3.697009,-1.090785e+06,4.643700e+06,7,828.760523,1946.941168
516,40.400245,-3.695893,-1.090685e+06,4.643700e+06,6,801.781734,1957.186734
517,40.400433,-3.694777,-1.090585e+06,4.643700e+06,4,786.679028,1972.455300
518,40.400620,-3.693661,-1.090485e+06,4.643700e+06,3,784.138915,1992.631404


OK. Let us now filter those locations: we're interested only in locations with no more than two restaurants in radius of 250 meters, and no Japanese restaurants in radius of 400 meters.

In [142]:
good_res_count = np.array((df_roi_locations['Restaurants nearby']<=2))
print('Locations with no more than two restaurants nearby:', good_res_count.sum())

good_jap_distance = np.array(df_roi_locations['Distance to Japanese restaurant']>=400)
print('Locations with no Japanese restaurants within 400m:', good_jap_distance.sum())

good_locations = np.logical_and(good_res_count, good_jap_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]


Locations with no more than two restaurants nearby: 120
Locations with no Japanese restaurants within 400m: 342
Locations with both conditions met: 107


Let's see how this looks on a map.

In [143]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

map_madrid = folium.Map(location=madrid_center, zoom_start=14)
folium.Marker(madrid_center, popup='Centro (Kilometro Cero)').add_to(map_madrid)

for i in range(1,len(districts.index)):
    folium.Marker([districts.Latitude[i], districts.Longitude[i]], popup=districts.District[i]).add_to(map_madrid)
    
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=300, color='yellow', fill=False).add_to(map_madrid)
        
folium.GeoJson(madrid_districts, style_function=districts_style, name='geojson').add_to(map_madrid)

HeatMap(df_restaurants.drop(["Name", "Adresses"], axis=1).values.tolist()).add_to(map_madrid)

folium.Circle(madrid_center, radius=2000, fill=False, color='white').add_to(map_madrid)

for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='green', fill=True, fill_color='green', fill_opacity=1).add_to(map_madrid) 

map_madrid

We have now is a clear indication of zones with low number of restaurants in vicinity, and *no* Japanese restaurants at all nearby. We can see that this are in Arganzuela and Retiro.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [144]:
from sklearn.cluster import KMeans

number_of_clusters = 5

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

In [145]:
map_madrid = folium.Map(location=madrid_center, zoom_start=13)
folium.Marker(madrid_center, popup='Centro (Kilometro Cero)').add_to(map_madrid)

for i in range(1,len(districts.index)):
    folium.Marker([districts.Latitude[i], districts.Longitude[i]], popup=districts.District[i]).add_to(map_madrid)
    
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=300, color='yellow', fill=False).add_to(map_madrid)
        
folium.GeoJson(madrid_districts, style_function=districts_style, name='geojson').add_to(map_madrid)

HeatMap(df_restaurants.drop(["Name", "Adresses"], axis=1).values.tolist()).add_to(map_madrid)

folium.Circle(madrid_center, radius=2000, fill=False, color='white').add_to(map_madrid)

for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='green', fill=True, fill_color='green', fill_opacity=1).add_to(map_madrid)
    
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='black', fill=True, fill_opacity=0.25).add_to(map_madrid) 


map_madrid

Finaly, let's reverse geocode those candidate area centers to get the addresses which can be presented to stakeholders.

In [146]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    geolocator = Nominatim(user_agent="foursquare_agent")
    addr = geolocator.reverse(lat, lon)
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, madrid_center_x, madrid_center_y)
    print('{}{} => {:.1f}km from Kilometro Cero'.format(addr, ' '*(50-len(addr)), d/1000))
    
    

Addresses of centers of areas recommended for further analysis

Barranc d'en Cabrera, Albocàsser, l'Alt Maestrat, Castelló / Castellón, Comunitat Valenciana, España                                                 => 1.7km from Kilometro Cero
Barranc d'en Cabrera, Albocàsser, l'Alt Maestrat, Castelló / Castellón, Comunitat Valenciana, España                                                 => 1.4km from Kilometro Cero
Povets, Albocàsser, l'Alt Maestrat, Castelló / Castellón, Comunitat Valenciana, España                                                 => 1.7km from Kilometro Cero
Albocàsser, l'Alt Maestrat, Castelló / Castellón, Comunitat Valenciana, España                                                 => 1.8km from Kilometro Cero
Barranc d'en Cabrera, Albocàsser, l'Alt Maestrat, Castelló / Castellón, Comunitat Valenciana, España                                                 => 1.7km from Kilometro Cero


This concludes our analysis. We have created 3 addresses (3 of them are the same address because the clusters are too close to each other) representing centers of zones containing locations with low number of restaurants and no Japanese restaurants nearby, all zones being fairly close to city center (all less than 2km from Kilometro Zero). Although zones are shown on the map as circles, their shape is actually very irregular and their centers/addresses should be considered only as a starting point for exploring area neighborhoods in search for potential restaurant locations. The zones found are located in Arganzuela and Retiro districts, which are very close to city center.

## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number of restaurants in Madrid (~1500 within 3km from Kilometro Cero), there are pockets of low restaurant density fairly close to city center. From the analysis of heatmaps, we decided to focus on the districts Centro, Retiro and Arganzuela, since these had a lesser concentration of restaurants.

After directing our attention to this more narrow area of interest (within 2km from Kilometro Cero) we first created a dense grid of location candidates (spaced 100m appart); those locations were then filtered so that those with more than two restaurants in radius of 250m and those with an Japanese restaurant closer than 400m were removed.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is 3 zones in Arganzuela and Retiro (we found that Centro was not a good option) containing largest number of potential new restaurant locations based on number of and distance to existing venues - both restaurants in general and Japanese restaurants particularly. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas close to Madrid center but not crowded with existing restaurants (particularly Japanese) - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Madrid areas close to center with low number of restaurants (particularly Japanese restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Japanese restaurant. By calculating restaurant density distribution from Foursquare data we have first identified general districts that justify further analysis (Centro, Retiro and Arganzuela), and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders. All of these points belong to Retiro and Arganzuela.

Final decision on optimal restaurant location will be made by stakeholders based on specific characteristics of districts and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.